# Predicting Heart Disease using Machine Learning 

This notebook uses various Python based machine learning and data science libraries in an attempt to build a machine learning model capable of predicting whether or not someone has a Heart Disease based on their medical attributes.
We're going to take the following approach:
1.	Problem Definition
2.	Data
3.	Evaluation
4.	Features
5.	Modelling
6.	Experimentation



    1. Problem Definition
In a statement,
Given clinical parameters about a patient, can we predict whether or not a they have a heart disease or not?


    2. Data

    Heart Disease UCI - Kaggle Version

    3.Evaluation
If we can reach 85% of accuracy at predicting whether or not a patient has heart disease during the proof of concept, we'll pursue the project.

    4.Features
The following are the features we'll use to predict our target variable (heart disease or no heart disease).


In [2]:
import pandas as pd
dataset=pd.read_csv('heart.csv')
dataset.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


Features



The following are the features we'll use to predict our target variable (heart disease or no heart disease).

1.	age - age in years

2.	sex - (1 = male; 0 = female)

3.	cp - chest pain type
    *	0: Typical angina: chest pain related decrease blood supply to the heart
    *   1: Atypical angina: chest pain not related to heart
    *   2: Non-anginal pain: typically esophageal spasms (non heart related)
    *	3: Asymptomatic: chest pain not showing signs of disease

4.	trestbps - resting blood pressure (in mm Hg on admission to the hospital)
    
    
        •*	anything above 130-140 is typically cause for concern

5.	chol - serum cholestoral in mg/dl
       
       •*	serum = LDL + HDL + .2 * triglycerides
       •*	above 200 is cause for concern

6.	fbs - (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
        
        *•	'>126' mg/dL signals diabetes

7.	restecg - resting electrocardiographic results
            
           *•	0: Nothing to note
        *•	1: ST-T Wave abnormality
            *	can range from mild symptoms to severe problems
            *	signals non-normal heart beat
        •*	2: Possible or definite left ventricular hypertrophy
                *	Enlarged heart's main pumping chamber

8.	thalach - maximum heart rate achieved

9.	exang - exercise induced angina (1 = yes; 0 = no)

10.	oldpeak - ST depression induced by exercise relative to rest
    
        •	looks at stress of heart during excercise
        •	unhealthy heart will stress more

11.	slope - the slope of the peak exercise ST segment
    
        •	0: Upsloping: better heart rate with excercise (uncommon)
        •	1: Flatsloping: minimal change (typical healthy heart)
        •	2: Downslopins: signs of unhealthy heart

12.	ca - number of major vessels (0-3) colored by flourosopy

        •	colored vessel means the doctor can see the blood passing through
        •	the more blood movement the better (no clots)

13.	thal - thalium stress result
        
        •	1,3: normal
        •	6: fixed defect: used to be defect but ok now
        •	7: reversable defect: no proper blood movement when excercising

14.	target - have disease or not (1=yes, 0=no) (= the predicted attribute)



#Importing Libraries

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.tree import DecisionTreeClassifier
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore")


#Assigning the input and output variables and checking value count of Output variable

In [4]:
X=dataset.drop('output',axis =1)#input variable
y=dataset['output']#output variable
y.value_counts()

1    165
0    138
Name: output, dtype: int64

#Feature Selection (Data- Categorical)

In [6]:
# statistical way to recall glm, by importing statsmodels.api as sm
model_logistic= sm.GLM(y, X, family=sm.families.Binomial()) # logistic regression
res = model_logistic.fit() # Fitting the model
res.summary()        

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                 output   No. Observations:                  303
Model:                            GLM   Df Residuals:                      290
Model Family:                Binomial   Df Model:                           12
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -106.64
Date:                Sun, 17 Apr 2022   Deviance:                       213.27
Time:                        12:00:22   Pearson chi2:                     279.
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
age            0.0128      0.019      0.670      0.503      -0.025       0.050
sex           -1.6381      0.452     -3.625      0.000      -2.524      -0.752
cp             0.8490      0.184      4.613      0.000       0.488       1.210
trtbps        -0.0153      0.010     -1.562      0.118      -0.035       0.004
chol          -0.0036      0.004     -0.960      0.337      -0.011       0.004
fbs           -0.0115      0.526     -0.022      0.983      -1.042       1.019
restecg        0.5432      0.342      1.589      0.112      -0.127       1.213
thalachh       0.0319      0.008      3.779      0.000       0.015       0.048
exng          -0.8920      0.403     -2.215      0.027      -1.681      -0.103
oldpeak       -0.4988      0.209     -2.381      0.017      -0.909      -0.088
slp            0.6092      0.346      1.761      0.078      -0.069       1.287
caa           -0.7725      0.189     -4.080      0.000      -1.144      -0.401
thall         -0.8438      0.287     -2.937      0.003      -1.407      -0.281
==============================================================================
"""

#if p_value<0.05 the feature is selected
#Selected features are-** sex, cp, thalachh, exng, oldpeak, caa, thall:**

# Feature scaling using MinMax Scaler

In [7]:
#out of selected features- sex, cp, exng, caa, thall, are categorical
#continuous features are- thalachh, oldpeak
categorical_cols = ['sex', 'cp',  'exng', 'caa','thall', 'output'] 
continuous_cols = [ 'thalachh', 'oldpeak'] 
label_col = ['output']

In [13]:
# Applying min-max scaler for continious columns
Heart_Cat = dataset[categorical_cols]
Heart_Con = MinMaxScaler().fit_transform(dataset[continuous_cols])
Heart_Con = pd.DataFrame(Heart_Con)
Heart_Con.rename(columns = { 0:'thalachh', 1:'oldpeak'}, inplace = True) # Renaming the columns

New_Heart = pd.merge(Heart_Con, Heart_Cat, left_index=True, right_index=True) # Merging the continious columns with categorical columns
New_Heart.head()# Displaying top 5 rows of new data

,thalachh,oldpeak,sex,cp,exng,caa,thall,output
0,0.603053,0.370968,1,3,0,0,1,1
1,0.885496,0.564516,1,2,0,0,2,1
2,0.770992,0.225806,0,1,0,0,2,1
3,0.816794,0.129032,1,1,0,0,2,1
4,0.702290,0.096774,0,0,1,0,2,1


# Assigning input and output variables from new data

In [15]:
x1=New_Heart.drop('output',axis =1)#input variable
y1=New_Heart['output']#output variable
y1.value_counts()

1    165
0    138
Name: output, dtype: int64

# Selecting Model  (Logistic, SVC, GaussianNB)

In [22]:
#Function-for getting models
def get_models():
    models=dict()

    models['lr']=LogisticRegression()   
    models['svc1']=SVC(kernel='linear')  
    models['svc2']=SVC(kernel='poly')
    models['svc3']=SVC(kernel='sigmoid')
    models['svc4']=SVC()
    models['gnb']=GaussianNB()
  
    return models

#Applying 1000 MonteCarlo Runs

In [39]:
#Function- for evaluating models
def evaluate_model(model, X, y):
    acc = [] 
    rec=[]
    for i in range(1000):
        X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)
        m = model.fit(X_train,y_train) # fit the model to trainset
        YPred = m.predict(X_test) # predict the output variable in the testset
        a = accuracy_score(y_test,YPred) #finding the accuracy
        acc.append(a) #appending the accuracy for every run
        cm=confusion_matrix(y_test,YPred)
        tp=cm[1][1]
        fn=cm[1][0]
        recall=tp/(tp+fn) 
        rec.append(recall)
        return np.mean(acc), np.mean(rec)

In [63]:
models = get_models()

results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, x1, y1)

    print((name, scores))

('lr', (0.8241758241758241, 0.9122807017543859))
('svc1', (0.8131868131868132, 0.9375))
('svc2', (0.8021978021978022, 0.9433962264150944))
('svc3', (0.7142857142857143, 0.6530612244897959))
('svc4', (0.8021978021978022, 0.8163265306122449))
('gnb', (0.7692307692307693, 0.8604651162790697))


# As per the accuracy and recall Logistic regression model is best for predicting the heart attack

#Importing the Pickle file

In [66]:
import pickle

#Dumping the LR model into pickle file

In [67]:
filename='heartattackprediction.pkl'
pickle.dump(models['lr'],open(filename,'wb'))